# Cardelli, Clayton, & Mathis 1989 interstellar extinction curve

Figure 6.3 from Chapter 6 of *Interstellar and Intergalactic Medium* by Ryden & Pogge, 2021, 
Cambridge University Press.

Plot the [Cardelli, Clayton, & Mathis 1989, ApJ, 345, 245](https://ui.adsabs.harvard.edu/abs/1989ApJ...345..245C/abstract) interstellar extinction curve, using the
updated optical/IR coefficients of [O'Donnell 1994, ApJ, 422, 158](https://ui.adsabs.harvard.edu/abs/1994ApJ...422..158O/abstract).

We use the parameterization of the extinction curve as a set of polynomials of order n.  

In [ ]:
%matplotlib inline

import os
import sys
import math
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, LogLocator, NullFormatter

import warnings
warnings.filterwarnings('ignore',category=UserWarning, append=True)

## Standard Plot Format

Setup the standard plotting format and make the plot. Fonts and resolution adopted follow CUP style.

In [ ]:
figName = 'Fig6_3' 

# graphic aspect ratio = width/height

aspect = 4.0/3.0 # 4:3

# Text width in inches - don't change, this is defined by the print layout

textWidth = 6.0 # inches

# output format and resolution

figFmt = 'png'
dpi = 600

# Graphic dimensions 

plotWidth = dpi*textWidth
plotHeight = plotWidth/aspect
axisFontSize = 10
labelFontSize = 6
lwidth = 0.5
axisPad = 5
wInches = textWidth 
hInches = wInches/aspect

# Plot filename

plotFile = f'{figName}.{figFmt}'

# LaTeX is used throughout for markup of symbols, Times-Roman serif font

plt.rc('text', usetex=True)
plt.rc('font', **{'family':'serif','serif':['Times-Roman'],'weight':'bold','size':'16'})

# Font and line weight defaults for axes

matplotlib.rc('axes',linewidth=lwidth)
matplotlib.rcParams.update({'font.size':axisFontSize})

# axis and label padding

plt.rcParams['xtick.major.pad'] = f'{axisPad}'
plt.rcParams['ytick.major.pad'] = f'{axisPad}'
plt.rcParams['axes.labelpad'] = f'{axisPad}'

## Extinction curve calculation

Compute the extinction curve for $R_V=3.1$ following CCM89 Equation 1:
\begin{equation}
   A_\lambda/A_V = a(x) + b(x)/R_V
\end{equation}
where $x=1/\lambda$, broken into three wavelength regions: IR, Optical/IR, and UV.

We adopt the O'Donnell polynomial coefficients $a(x)$ and $b(x)$ for the Optical/IR regime, otherwise use the
CCM89 parameterization.


In [ ]:
# Ratio of total to selective extinction

Rv = 3.1

# x has units of 1/um

# Infrared: x=0.3 to 1.1/um: CCM89 Eqn 2a and 2b

xIR = np.linspace(0.3,1.1,100)
aIR =  0.574*(xIR**1.61)
bIR = -0.527*(xIR**1.61)

AlamIR = aIR + bIR/Rv

# Optical/NIR: x=1.1 to 3.3/um and y=x-1.82 with the O'Donnel 1994 coefficients

xOIR = np.linspace(1.1,3.3,100)
y = xOIR-1.82

ca = [1.0,0.104,-0.609, 0.701, 1.137,-1.718,-0.827,  1.647,-0.505]
cb = [0.0,1.952, 2.908,-3.989,-7.985,11.102, 5.491,-10.805, 3.347]

aOIR = np.polynomial.polynomial.polyval(y,ca)
bOIR = np.polynomial.polynomial.polyval(y,cb)

AlamOIR = aOIR + bOIR/Rv

# Mid-UV

xMUV = np.linspace(3.3,8.0,100)

aMUV = np.zeros(100)
bMUV = np.zeros(100)
for i, x in enumerate(xMUV):
    if x < 5.9:
        Fa = 0.0
        Fb = 0.0
    else:
        Fa = -0.04473*(x-5.9)**2 - 0.009779*(x-5.9)**3
        Fb = 0.2130*(x-5.9)**2 + 0.1207*(x-5.9)**3
    aMUV[i] = 1.752 - 0.316*x -0.104/((x-4.67)**2 + 0.341) + Fa
    bMUV[i] = -3.090 + 1.825*x + 1.206/((x-4.62)**2 + 0.263) + Fb

AlamMUV = aMUV + bMUV/Rv

# Far-UV

xFUV = np.linspace(8.0,10.0,100)
y = xFUV - 8.0
ca = [-1.073,-0.628, 0.137,-0.070 ]
cb = [13.670, 4.257,-0.420, 0.374 ]

aFUV = np.polynomial.polynomial.polyval(y,ca)
bFUV = np.polynomial.polynomial.polyval(y,cb)

AlamFUV = aFUV + bFUV/Rv


## Plot the extinction curve

In [ ]:
xMin = 0.1
xMax = 1.0
yMin = 0.0
yMax = 5.5

fig,ax = plt.subplots(figsize=(wInches,hInches),dpi=dpi)

ax.tick_params('both',length=6,width=lwidth,which='major',direction='in',top='on',right='on')
ax.tick_params('both',length=3,width=lwidth,which='minor',direction='in',top='on',right='on')

# Limits

ax.set_xlim(xMin,xMax)
ax.xaxis.set_major_locator(MultipleLocator(0.1))
ax.xaxis.set_minor_locator(MultipleLocator(0.05))
ax.set_xlabel(r'$\lambda$ [$\mu$m]',fontsize=axisFontSize)

ax.set_ylim(yMin,yMax)
ax.yaxis.set_major_locator(MultipleLocator(1))
ax.yaxis.set_minor_locator(MultipleLocator(0.5))
ax.set_ylabel(r'A$_\lambda$/A$_V$',fontsize=axisFontSize)


ax.plot(1./xIR,AlamIR,color='black',ls='-',lw=1.5,zorder=10)
ax.plot(1./xOIR,AlamOIR,color='black',ls='-',lw=1.5,zorder=10)
ax.plot(1./xMUV,AlamMUV,color='black',ls='-',lw=1.5,zorder=10)
ax.plot(1./xFUV,AlamFUV,color='black',ls='-',lw=1.5,zorder=10)

ax.vlines([0.4,0.7],yMin,yMax,ls=['--'],colors=['black'],lw=0.5,zorder=8)

ax.text((0.1+0.4)/2.0,5,r'UV',fontsize=axisFontSize,ha='center',color='black')
ax.text((0.4+0.7)/2.0,5,r'Visible',fontsize=axisFontSize,ha='center',color='black')
ax.text((0.7+1.0)/2.0,5,r'Infrared',fontsize=axisFontSize,ha='center',color='black')

plt.plot()
plt.savefig(plotFile,bbox_inches='tight')